# common

In [ ]:
//// test

open testing

()



## types

In [ ]:
inl types () =
    env.types ()
    rust.types ()
    sm'.types ()

()



## prototype

In [ ]:
prototype (~:>) r : forall t. t -> r

()



## run_target

In [ ]:
//// test

run_target function
    | Fsharp (Native) => fun () => $'1uy'
    | _ => fun () => $'2uy'
|> _assert_eq 1u8

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : bool = true
    let mutable _v0 : uint8 option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v1 : uint8 = 2uy
    v1 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v2 : uint8 = 2uy
    v2 
    #endif
#if FABLE_COMPILER_RUST && CONTRACT
    let v3 : uint8 = 2uy
    v3 
    #endif
#if FABLE_COMPILER_TYPESCRIPT
    let v4 : uint8 = 2uy
    v4 
    #endif
#if FABLE_COMPILER_PYTHON
    let v5 : uint8 = 2uy
    v5 
    #endif
#else
    let v6 : uint8 = 1uy
    v6 
    #endif
    |> fun x -> _v0 <- Some x
    let v7 : uint8 = _v0.Value
    let v8 : bool = v7 = 1uy
    let v10 : bool =
        if v8 then
            true
        else
            method1(v8)
    let v11 : string = "assert_eq"
    let v12 : string = $"{v11} / actual: %A{v7} / expected: %A{1uy}"
    System.Console.WriteLine v12
    let v13 : bool = v10 = false
    if v13 then
     

In [ ]:
//// test
///! rust

run_target function
    | Rust (Native) => fun () => $'1uy'
    | _ => fun () => $'2uy'
|> _assert_eq 1u8

.rs output:
assert_eq / actual: 1 / expected: 1




.fsx:
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : bool = true
    let mutable _v0 : uint8 option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v1 : uint8 = 1uy
    v1 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v2 : uint8 = 2uy
    v2 
    #endif
#if FABLE_COMPILER_RUST && CONTRACT
    let v3 : uint8 = 2uy
    v3 
    #endif
#if FABLE_COMPILER_TYPESCRIPT
    let v4 : uint8 = 2uy
    v4 
    #endif
#if FABLE_COMPILER_PYTHON
    let v5 : uint8 = 2uy
    v5 
    #endif
#else
    let v6 : uint8 = 2uy
    v6 
    #endif
    |> fun x -> _v0 <- Some x
    let v7 : uint8 = _v0.Value
    let v8 : bool = v7 = 1uy
    let v10 : bool =
        if v8 then
            true
        else
            method1(v8)
    let v11 : string = "assert_eq"
    let v12 : string = $"{v11} / actual: %A{v7} / expected: %A{1uy}"
    System.Console.WriteLine v12
    let v13 : bool = v10 = false
    if v13 then

## upcast

In [ ]:
inl upcast forall t u. (x : t) : u =
    $'!x :> `u '

()



## downcast

In [ ]:
inl downcast forall t u. (x : t) : u =
    $'!x :?> `u '

()



## memoize

In [ ]:
nominal lazy t = $'Lazy<`t>'

inl memoize forall t. (fn : () -> t) : () -> t =
    inl fn = join fn
    inl result : lazy t = $'lazy !fn ()'
    fun () => $'!result.Value'

()



In [ ]:
//// test

inl count = mut 0i32
inl add =
    fun () => count <- *count + 1
    |> memoize

add ()
add ()
add ()

*count
|> _assert_eq 1

type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : (unit -> unit) = method1(v0)
    let v2 : Lazy<unit> = lazy v1 ()
    v2.Value
    v2.Value
    v2.Value
    let v3 : int32 = v0.l0
    let v4 : bool = v3 = 1
    let v6 : bool =
        if v4 then
            true
        else
            method2(v4)
    let v7 : string = "assert_eq"
    let v8 : string = $"{v7} / actual: %A{v3} / expected: %A{1}"
    System.Console.WriteLine v8
    let v9 : bool = v6 = false
    if v9 then
        failwith<unit> v8
method0()

assert_eq / actual: 1 / expected: 1


## to_any

In [ ]:
inl to_any forall t. (obj : t) : any =
    $"!obj"

instance (~:>) any = to_any

()



## (||>)

In [ ]:
//// test

(3i32, 2i32)
||> fun a b => a - b
|> _assert_eq 1

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{1} / expected: %A{1}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: 1 / expected: 1


## flip

In [ ]:
inl flip fn a b =
    fn b a

()



In [ ]:
//// test

(1i32, 2i32)
||> flip pair
|> _assert_eq (2, 1)

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{struct (2, 1)} / expected: %A{struct (2, 1)}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: struct (2, 1) / expected: struct (2, 1)


## join_body

In [ ]:
inl join_body body acc x =
    if var_is x |> not
    then body acc x
    else
        inl acc = dyn acc
        join body acc x

()



In [ ]:
//// test

inl rec fold_list f s = function
    | Cons (x, x') => fold_list f (f s x) x'
    | Nil => s

()



In [ ]:
//// test

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (+) 0
|> _assert_eq 15

let rec method1 () : int32 =
    3
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = 9 + v0
    let v2 : int32 = v1 + 2
    let v3 : int32 = v2 + 1
    let v4 : bool = v3 = 15
    let v6 : bool =
        if v4 then
            true
        else
            method2(v4)
    let v7 : string = "assert_eq"
    let v8 : string = $"{v7} / actual: %A{v3} / expected: %A{15}"
    System.Console.WriteLine v8
    let v9 : bool = v6 = false
    if v9 then
        failwith<unit> v8
method0()

assert_eq / actual: 15 / expected: 15


In [ ]:
//// test

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (join_body (+)) 0
|> _assert_eq 15

let rec method1 () : int32 =
    3
and method2 (v0 : int32, v1 : int32) : int32 =
    let v2 : int32 = v1 + v0
    v2
and method3 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = 9
    let v2 : int32 = method2(v0, v1)
    let v3 : int32 = v2 + 2
    let v4 : int32 = v3 + 1
    let v5 : bool = v4 = 15
    let v7 : bool =
        if v5 then
            true
        else
            method3(v5)
    let v8 : string = "assert_eq"
    let v9 : string = $"{v8} / actual: %A{v4} / expected: %A{15}"
    System.Console.WriteLine v9
    let v10 : bool = v7 = false
    if v10 then
        failwith<unit> v9
method0()

assert_eq / actual: 15 / expected: 15


## join_body_unit

In [ ]:
inl join_body_unit body d x =
    if var_is d |> not
    then body x
    else
        inl x = dyn x
        join body x

()



In [ ]:
//// test

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (fun acc n => join_body_unit ((+) acc) n n) 0
|> _assert_eq 15

let rec method1 () : int32 =
    3
and method2 (v0 : int32) : int32 =
    let v1 : int32 = 9 + v0
    v1
and method3 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = method2(v0)
    let v2 : int32 = v1 + 2
    let v3 : int32 = v2 + 1
    let v4 : bool = v3 = 15
    let v6 : bool =
        if v4 then
            true
        else
            method3(v4)
    let v7 : string = "assert_eq"
    let v8 : string = $"{v7} / actual: %A{v3} / expected: %A{15}"
    System.Console.WriteLine v8
    let v9 : bool = v6 = false
    if v9 then
        failwith<unit> v8
method0()

assert_eq / actual: 15 / expected: 15


## nameof

In [ ]:
inl nameof x : string =
    $"nameof !x"

()



In [ ]:
//// test

fun () => failwith "test"
|> _throws
|> optionm.map sm'.format_exception
|> _assert_eq (Some "System.Exception: test")

type [<Struct>] US0 =
    | US0_0 of f0_0 : exn
    | US0_1
and [<Struct>] US1 =
    | US1_0 of f0_0 : string
    | US1_1
let rec closure0 () () : unit =
    failwith<unit> "test"
and closure1 () (v0 : exn) : US0 =
    US0_0(v0)
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> unit) = closure0()
    let v1 : US0 = US0_1
    let v2 : (exn -> US0) = closure1()
    let v3 : US0 = try v0 (); v1 with ex -> ex |> v2
    let v16 : US1 =
        match v3 with
        | US0_1 -> (* None *)
            US1_1
        | US0_0(v4) -> (* Some *)
            let v5 : bool = true
            let mutable _v5 : string option = None 
            
#if FABLE_COMPILER || WASM || CONTRACT
            
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
            let v6 : string = $"%A{v4}"
            v6 
            #endif
#if FABLE_COMPILER_RUST && WASM
            let v7 : string = $"%A{v4}"
            v7 
            #endif
#if FABLE_COMPILER_RUST && CONTRACT
           

## disposable

In [ ]:
nominal disposable t = $"System.IDisposable"

()



## dispose

In [ ]:
inl dispose (disposable : disposable _) : () =
    disposable |> $'_.Dispose()'

()



## new_disposable

In [ ]:
inl new_disposable (fn : () -> ()) : disposable _ =
    run_target function
        | Rust _ => fun () =>
            global "type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()"
            $'new Disposable (fun () -> Fable.Core.RustInterop.emitRustExpr !fn "$0()" )'
        | Fsharp _ => fun () =>
            inl fn = join fn
            $'{ new System.IDisposable with member _.Dispose () = !fn () }'
        | _ => fun () => null ()

()



In [ ]:
//// test

inl new_disposable_test = mut 0i32
new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
|> fun x => x : disposable ()
|> dispose
*new_disposable_test |> _assert_eq 1

type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()
type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : bool = true
    let mutable _v1 : System.IDisposable option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : (unit -> unit) = closure0(v0)
    let v3 : System.IDisposable = new Disposable (fun () -> Fable.Core.RustInterop.emitRustExpr v2 "$0()" )
    v3 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v4 : System.IDisposable = new Disposable (fun () -> Fable.Core.RustInterop.emitRustExpr v2 "$0()" )
    v4 
    #endif
#if FABLE_COMPILER_RUST && CONTRACT
    let v5 : System.IDisposable = new Disposable 

In [ ]:
//// test
//// print_code=false
///! rust

inl new_disposable_test = mut 0i32
new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
|> fun x => x : disposable ()
|> dispose
*new_disposable_test |> _assert_eq 1

assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_task
|> async.await_task
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()
type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : bool = true
    let mutable _v1 : System.Threading.Tasks.Task option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : System.Threading.Tasks.Task = null |> unbox<System.Threading.Tasks.Task>
    v2 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v3 : System.Threading.Tasks.Task = null |> unbox<System.Threading.Tasks.Task>
    v3 
    #endif
#if FABLE_COMPILER_RUST && CONTRACT
    let v4 : System.Threading.Tasks.Task = null |> unbox<System.Threading.Tasks.Task>
    v4 
    #endif
#if FABLE_COMPILER_TYP

In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()
type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : bool = true
    let mutable _v1 : Async<unit> option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : Async<unit> = null |> unbox<Async<unit>>
    v2 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v3 : Async<unit> = null |> unbox<Async<unit>>
    v3 
    #endif
#if FABLE_COMPILER_RUST && CONTRACT
    let v4 : Async<unit> = null |> unbox<Async<unit>>
    v4 
    #endif
#if FABLE_COMPILER_TYPESCRIPT
    let v5 : Async<unit> = null |> unbox<Async<unit>>
    v5 
    #endif
#if FABLE_COMPILER_PYTHON
    l

In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 0

type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()
type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : bool = true
    let mutable _v1 : Async<unit> option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : Async<unit> = null |> unbox<Async<unit>>
    v2 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v3 : Async<unit> = null |> unbox<Async<unit>>
    v3 
    #endif
#if FABLE_COMPILER_RUST && CONTRACT
    let v4 : Async<unit> = null |> unbox<Async<unit>>
    v4 
    #endif
#if FABLE_COMPILER_TYPESCRIPT
    let v5 : Async<unit> = null |> unbox<Async<unit>>
    v5 
    #endif
#if FABLE_COMPILER_PYTHON
    l

## return

In [ ]:
inl return forall t. (x : t) : () =
    $'return !x '

inl return' forall t. (x : t) : t =
    $'return !x '

()



## sleep

In [ ]:
inl sleep (n : i32) : () =
    run_target function
        | Fsharp (Native) => fun () => $'System.Threading.Thread.Sleep' n
        | _ => fun () => ()

()



## retry_fn

In [ ]:
inl retry_fn forall t. retries (fn : () -> t) : option t =
    let rec loop retry =
        try
            fun () =>
                if retry < retries
                then fn () |> Some
                else None
            fun ex =>
                trace Warning
                    fun () => "retry_fn"
                    fun () => $'$"retry: {!retry} / ex: %A{!ex} / {!_locals ()}"'
                sleep 1
                None
        |> function
            | Some x => x
            | None => loop (retry + 1)
    loop 0

()



In [ ]:
//// test

inl retry_fn_test = mut 0i32
fun () =>
    retry_fn_test <- *retry_fn_test + 1
    *retry_fn_test
|> retry_fn 3i32
|> _assert_eq (Some 1i32)

module State = let mutable trace_state = None
type Mut0 = {mutable l0 : int32}
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
and [<Struct>] US1 =
    | US1_0 of f0_0 : US0
    | US1_1
and [<Struct>] US2 =
    | US2_0
    | US2_1
    | US2_2
    | US2_3
    | US2_4
and Mut1 = {mutable l0 : int64}
and Mut2 = {mutable l0 : bool}
and Mut3 = {mutable l0 : US2}
and Mut4 = {mutable l0 : (string -> unit)}
and [<Struct>] US3 =
    | US3_0 of f0_0 : int64
    | US3_1
let rec closure0 (v0 : Mut0, v1 : int32) () : US0 =
    let v2 : bool = v1 < 3
    if v2 then
        let v3 : int32 = v0.l0
        let v4 : int32 = v3 + 1
        v0.l0 <- v4
        let v5 : int32 = v0.l0
        US0_0(v5)
    else
        US0_1
and closure1 () (v0 : US0) : US1 =
    US1_0(v0)
and closure3 () () : string =
    let v0 : string = "retry_fn"
    v0
and closure5 () () : string =
    let v0 : string = ""
    v0
and closure4 (v0 : int32, v1 : exn) () : string =
    let v2 : (unit -> string) = closure5()


In [ ]:
//// test

inl retry_fn_test = mut 0i32
fun () =>
    if *retry_fn_test >= 2
    then *retry_fn_test
    else
        retry_fn_test <- *retry_fn_test + 1
        failwith "test"
|> retry_fn 3i32
|> _assert_eq (Some 2i32)

module State = let mutable trace_state = None
type Mut0 = {mutable l0 : int32}
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
and [<Struct>] US1 =
    | US1_0 of f0_0 : US0
    | US1_1
and [<Struct>] US2 =
    | US2_0
    | US2_1
    | US2_2
    | US2_3
    | US2_4
and Mut1 = {mutable l0 : int64}
and Mut2 = {mutable l0 : bool}
and Mut3 = {mutable l0 : US2}
and Mut4 = {mutable l0 : (string -> unit)}
and [<Struct>] US3 =
    | US3_0 of f0_0 : int64
    | US3_1
let rec closure0 (v0 : Mut0, v1 : int32) () : US0 =
    let v2 : bool = v1 < 3
    if v2 then
        let v3 : int32 = v0.l0
        let v4 : bool = v3 >= 2
        let v9 : int32 =
            if v4 then
                let v5 : int32 = v0.l0
                v5
            else
                let v6 : int32 = v0.l0
                let v7 : int32 = v6 + 1
                v0.l0 <- v7
                failwith<int32> "test"
        US0_0(v9)
    else
        US0_1
and closure1 () (v0 : US0) : US1 =
    US1_0(v0)
and clo

## main

In [ ]:
inl main () =
    types ()
    init_trace_state ()
    inl new_disposable x : _ () = new_disposable x
    $"let new_disposable x = !new_disposable x" : ()

    inl retry_fn (r : i32) (x : () -> _) : optionm'.option' () = retry_fn r x |> optionm'.box
    $"let retry_fn x = !retry_fn x" : ()
    inl memoize (fn : () -> ()) : () -> () = memoize fn
    $"let memoize x = !memoize x" : ()

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::env::VarError")>]
#endif
type std_env_VarError = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("_")>]
#endif
type Any = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]